## As a dual citizen, I am curious which passport I should best use entering what country.

In [4]:
from bs4 import BeautifulSoup

In [48]:
german = "https://en.wikipedia.org/wiki/Visa_requirements_for_German_citizens"
polish = "https://en.wikipedia.org/wiki/Visa_requirements_for_Polish_citizens"

de_html = requests.get(german).text
pl_html = requests.get(polish).text

In [200]:
de_soup = BeautifulSoup(de_html, 'lxml')
de = pd.read_html(str(de_soup.find("table", {'class' : 'sortable wikitable'})), skiprows=1)[0]

pl_soup = BeautifulSoup(pl_html, 'lxml')
pl = pd.read_html(str(pl_soup.find("table", {'class' : 'sortable wikitable'})), skiprows=1)[0]

In [201]:
de.head()

,0,1,2,3
0,Afghanistan,Visa required[27],NaN,NaN
1,Albania,Visa not required[28],90 days,ID card valid
2,Algeria,Visa required[29],NaN,NaN
3,Andorra,Visa not required[30],NaN,ID card valid
4,Angola,eVisa[31][32],30 days,Visitors who were granted online pre-visa are ...


In [202]:
pl.head()

,0,1,2,3
0,Afghanistan,Visa required[2],NaN,NaN
1,Albania,Visa not required[3],90 days,ID card valid
2,Algeria,Visa required[4],NaN,NaN
3,Andorra,Visa not required[5],NaN,ID card valid
4,Angola,eVisa[6][7],30 days,Visitors who were granted online pre-visa are ...


In [203]:
# I am going to merge them into one dataframe to avoid running every function twice on each data frame
# and then separate them when I'm done

In [204]:
de.columns = ['country', 'status', 'duration', 'notes']
pl.columns = ['country', 'status', 'duration', 'notes']

de['cat'] = 'DE'
pl['cat'] = 'PL'

df = de.append(pl)

In [206]:
de.status.str.contains("\[").value_counts()

True    193
Name: status, dtype: int64

In [207]:
df.status = df.status.str.findall("^(.+?)\[").apply(lambda x: x[0] if len(x) > 0 else np.nan)

In [208]:
df.status.value_counts()

Visa not required          226
Visa required               69
Visa on arrival             32
eVisa / Visa on arrival     31
eVisa                       15
Free visa on arrival         6
Tourist Card required        2
eVisitor                     2
e-Visa                       1
Tourist Card on arrival      1
Visa Waiver Program          1
Name: status, dtype: int64

In [209]:
d = {'eVisa / Visa on arrival' : 'eVisa',
    'Visa Waiver Program' : 'Visa not required',
    'Tourist Card required' : 'Visa required',
    'Tourist Card on arrival' : 'Visa on arrival',
    'eVisitor' : 'e-Visa',
    'Free visa on arrival' : 'Visa on arrival'}

In [210]:
df.status.replace(d, inplace=True)

In [212]:
df.head()

,country,status,duration,notes,cat
0,Afghanistan,Visa required,NaN,NaN,DE
1,Albania,Visa not required,90 days,ID card valid,DE
2,Algeria,Visa required,NaN,NaN,DE
3,Andorra,Visa not required,NaN,ID card valid,DE
4,Angola,eVisa,30 days,Visitors who were granted online pre-visa are ...,DE


In [213]:
df.duration.fillna("0", inplace=True)

In [214]:
df.loc[df.duration.str.contains("month"), 'duration']

5          6 months
11         8 months
14         6 months
17          1 month
26          1 month
30         6 months
52         3 months
57         4 months
65         3 months
69         3 months
71         3 months
85         3 months
89         3 months
93          1 month
103        3 months
119        3 months
123        3 months
142        3 months
151        3 months
175        4 months
176        3 months
192        3 months
5      6 months[12]
11         3 months
14         3 months
17          1 month
26          1 month
30         6 months
41         3 months
52         3 months
57         4 months
66         3 months
72         3 months
80         3 months
86         3 months
90         3 months
94          1 month
104        3 months
124        3 months
142        3 months
151        3 months
169        3 months
175        3 months
176        3 months
192        3 months
Name: duration, dtype: object

In [215]:
# Converting months to days

df.loc[df.duration.str.contains("month"), 'duration'] = \
df.loc[df.duration.str.contains("month"), 'duration'].str.findall("([0-9])").apply(lambda x : str(int(x[0]) * 30))

In [ ]:
####

In [216]:
df.loc[df.duration.str.contains("days"), 'duration'] = \
df.loc[df.duration.str.contains("days"), 'duration'].str.findall("([0-9]+)\sdays").apply(lambda x: x[0])

In [220]:
df.loc[df.duration.str.contains("Freedom"), 'duration'] = '999'

In [221]:
df[df.duration.str.contains("[^0-9]")]

,country,status,duration,notes,cat
62,Georgia,Visa not required,1 year,ID card valid,DE
37,Comoros,Visa on arrival,2 weeks,NaN,PL
62,Georgia,Visa not required,1 year,ID card valid,PL


In [222]:
df.replace("1 year", "365", inplace=True)
df.replace("2 weeks", "14", inplace=True)

In [224]:
df.duration = df.duration.astype(int)

In [226]:
pl = df[df.cat == "PL"]
de = df[df.cat == "DE"]

In [228]:
len(de) == len(pl)

True

In [236]:
merged = pd.merge(pl, df, on="country", suffixes=('_PL', '_DE'))

In [237]:
merged[merged.duration_DE > merged.duration_PL]

,country,status_PL,duration_PL,notes_PL,cat_PL,status_DE,duration_DE,notes_DE,cat_DE
22,Bahamas,Visa not required,90,NaN,PL,Visa not required,240,NaN,DE
28,Barbados,Visa not required,90,NaN,PL,Visa not required,180,NaN,DE
139,Guyana,Visa required,0,NaN,PL,Visa not required,90,NaN,DE
163,Jamaica,Visa not required,30,NaN,PL,Visa not required,90,NaN,DE
189,Lesotho,eVisa,0,NaN,PL,Visa not required,14,NaN,DE
229,Mongolia,Visa required,0,NaN,PL,Visa not required,30,NaN,DE
239,Namibia,Visa required,0,NaN,PL,Visa not required,90,3 months within a calendar year,DE
315,South Africa,Visa not required,30,NaN,PL,Visa not required,90,NaN,DE
325,Suriname,Visa required,0,NaN,PL,Visa on arrival,90,NaN,DE
349,Tunisia,Visa not required,90,NaN,PL,Visa not required,120,ID card valid in certain cases,DE


In [238]:
merged[merged.duration_DE < merged.duration_PL]

,country,status_PL,duration_PL,notes_PL,cat_PL,status_DE,duration_DE,notes_DE,cat_DE
74,Comoros,Visa on arrival,14,NaN,PL,Visa on arrival,0,NaN,DE
82,Côte d'Ivoire,eVisa,90,eVisa holders must arrive via Port Bouet Airport.,PL,eVisa,0,NaN,DE
159,Israel,Visa not required,90,NaN,PL,Visa not required,0,3 months for tourism only.[125] German citizen...,DE
337,Tanzania,Visa on arrival,90,NaN,PL,Visa on arrival,0,NaN,DE
377,Vietnam,eVisa,30,NaN,PL,Visa not required,15,A single entry eVisa valid for 30 days is also...,DE


In [239]:
mask1 = merged.status_DE.str.contains("not required|eVisa")
mask2 = merged.status_PL.str.contains("not required|eVisa")

merged[mask1 * ~mask2]

/usr/local/lib/python3.5/dist-packages/pandas/computation/expressions.py:182: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


,country,status_PL,duration_PL,notes_PL,cat_PL,status_DE,duration_DE,notes_DE,cat_DE
139,Guyana,Visa required,0,NaN,PL,Visa not required,90,NaN,DE
149,India,e-Visa,60,e-Visa holders must arrive via 25 designated a...,PL,eVisa,60,e-Visa holders must arrive via 25 designated a...,DE
229,Mongolia,Visa required,0,NaN,PL,Visa not required,30,NaN,DE
239,Namibia,Visa required,0,NaN,PL,Visa not required,90,3 months within a calendar year,DE
365,United States,Visa required,0,NaN,PL,Visa not required,90,On arrival from overseas; ESTA (valid for 2 ye...,DE
